# Домашняя работа №2: Нейросетевое ранжирование и анализ историй трансформером

На семинарах были рассмотрены различные методы и идеи ручного составления признаков для задачи ранжирования. В этом семинаре мы посмотрим, как этот процесс можно автоматизировать с помощью применения трансформера над историей взаимодействий пользователя. Строить алгоритм будем на данных из вашего контеста.

## 1. Общая схема алгоритма

Алгоритм, который вы будете реализовывать, будет представлять собой применение трансформера к истории позитивных взаимодействий пользователя. В нашем случае это клики, корзины и покупки. Просмотры учитывать можно и нужно, но для упрощения задачи будем их игнорировать.

Обучение модели будет состоять из двух частей: pretrain и finetune. Pretrain представляет из себя обучение на некоторую общую задачу, которая максимально утилизирует все имеющиеся данные. Такие задачи, как правило, хорошо "масштабируются", то есть добавление данных и увеличение модели приводит к повышению качества. Finetune представляет из себя выравнивание модели с целевой задачей. Это, как правило, более разреженная задача. В нашем случае целевой метрикой является ndcg@10 по группам (request_id) на тесте. Для этого хорошо работает обучение на попарную функцию потерь в рамках этих групп. Мы будем использовать [Calibrated Pairwise Logistic](https://arxiv.org/pdf/2211.01494) (см. далее). Метки в группе будут 0 для просмотра и 1 для корзины. Клики и покупки не учитываются, т.к. их нет в итоговом тесте. Задача - отранжировать единички (корзины) как можно выше ноликов (просмотров).

Далее будет краткое описание каждой из задач, конкретные особенности в следующих разделах.

### Pretrain:

<div align="center">
  <img src="https://i.ibb.co/8GksnWD/Screenshot-2025-05-04-at-10-15-36.png" width="500" alt="pretrain">
</div>

Pretrain будет состоять из двух задач: next-positive prediction и feedback prediction.

#### Next-positive prediction

Концептуально задача заключаетя в предсказании следующего положительного взаимодействия пользователя на основе истории всех предыдущих положительных взаимодействий. У каждого положительного взаимодействия есть три характеристики:
- Товар, с которым было сделано взаимодействие
- Контекст, в котором было сделано взаимодействие (поверхность, время и т.п.)
- Фидбек этого взаимодействия (например, клик).

Каждую характеристику можно закодировать в виде вектора (про это далее). Модель получает историю в виде последовательности токенов, каждый из которых представляется в виде суммы $c_t, i_t, f_t$, где

- $c_t$ – контекст t-го взаимодействия,  
- $i_t$ – товар взаимодействия,  
- $f_t$ – наблюдаемый фидбэк (клик, корзина, покупка).

Задача:
$$
P(\text{item}=i_t\mid \text{history}=S_{t-1},\;\text{context}=c_t)\,.
$$

Используем softmax по множеству всех товаров для моделирования такой вероятности. Близость между товаром и парой (контекст, пользователь) считаем через косинус. Обучаться будем на кросс-энтропию.

#### Feedback prediction

Задача:
$$
P(\text{feedback}=f_t\mid \text{history}=S_{t-1},\;\text{context}=c_t,\;\text{item}=i_t).
$$

В нашем случае решаем задачу классификации на три класса: клик, корзина, покупка.

Итоговый pretrain лосс:
$$
\mathcal{L}_{\rm pre\text{-}train}
= \mathcal{L}_{\mathrm{NPP}} + \mathcal{L}_{\mathrm{FP}}.
$$

#### Позднее связывание

- Трансформер выдаёт по одному скрытому состоянию $h_t$ на каждое взаимодействие.  
- Для next-positive prediction конкатенируем вектор текущего скрытого состояния $h_{t-1}$ и следующего контекста $c_t$:
$$
\hat h^c_t = \mathrm{MLP}\bigl(\mathrm{Concat}(h_{t-1},c_t)\bigr).
$$
- Для feedback prediction дополнительно еще конкатенируем вектор следующего товара $i_t$ для предсказания следующего feedback-а:
$$
\hat h^i_t = \mathrm{MLP}\bigl(\mathrm{Concat}(h_{t-1},c_t,i_t)\bigr).
$$
- Позднее, потому что после применения трансформера.

### Finetune

<p align="center">
  <img src="https://i.ibb.co/kgwt7pRb/Screenshot-2025-05-04-at-10-15-48.png" width="500" alt="finetune">
</p>

#### Постановка задачи  
Пусть есть пользователь с идентификатором `user_id`. У него сформирована история взаимодействий и набор групп (каждая группа имеет свой `request_id`).  
- Скрытые состояния модели на момент каждого взаимодействия:  
  $$h_0, h_1, \dots, h_t$$  
- Historical impressions - это последовательность групп, в которых пользователь видел товары.  

Каждая группа представляет собой множество товаров c бинарными метками (таргеты) 0 или 1.  
Задача finetune - для каждого товара внутри группы оценить его релевантность пользователю, используя состояние пользователя, актуальное на момент показа этой группы.

#### Учет задержки (consistency with validation)  
На тесте между последним известным состоянием истории и новой группой может пройти от 2 дней до 1 месяца.  
Чтобы обучение было консистентно с валидацией, имитируем такой промежуток:

1. Случайно выбираем задержку для каждой группы
   $$\Delta \sim \mathrm{Uniform}(2\ \text{дн.},\ 32 \text{дн.}).$$
2. Находим первое состояние $h_k$, предшествующее текущей группе не менее чем на $\Delta$.  
3. Используем это состояние $h_k$ как вектор пользователя для расчёта потерь.

#### Попарная ранжирующая функция потерь  
В рамках каждой группы (`request_id`) формируем все возможные пары товаров $(i,j)$, где товар $i$ имеет таргет 1, а товар $j$ - таргет 0.  
Для каждой пары рассчитываем ранжирующую функцию потерь на основе предсказанных релевантностей.

## 2. Предобработка данных

In [ ]:
import polars as pl
from collections import deque
from typing import Dict, Any, Generator, Iterable, Optional
from abc import ABC, abstractmethod
from itertools import chain

Скачайте актуальные данные с https://www.kaggle.com/competitions/ysda-recsys-2025-lavka/data.  Оставляем только небольшой поднабор всех признаков. Учесть все имеющиеся признаки можно будет в бонусной части.

In [ ]:
train_df = pl.read_parquet('./data/train.parquet').select(['action_type', 'product_id', 'source_type', 'timestamp', 'user_id', 'request_id'])

### Разделение на трейн и валидация (повторяем распределение теста):

<div align="center">
  <img src="https://i.ibb.co/yBPn87t7/IMG000-19.jpg" width="500" alt="split">
</div>

Для оценки модели будем использовать train данные. Из них сформируем валидационную и обучающую части. Для того, чтобы получить корректные метрики на валидации, важно повторить все особенности тестовых данных:
- 2 дня разница между train и valid
- 1 месяц на valid
- оставляем только просмотр и корзину
- группы с >= 10 товарами
- группы с хотя бы одной корзиной и хотя бы одним просмотром

In [ ]:
class ActionType:
    VIEW = 'AT_View'
    CLICK = 'AT_Click'
    CART_UPDATE = 'AT_CartUpdate'
    PURCHASE = 'AT_Purchase'


class Preprocessor:
    mapping_action_types = {
        ActionType.VIEW: 0,
        ActionType.CART_UPDATE: 1,
        ActionType.CLICK: 2,
        ActionType.PURCHASE: 3
    }
    def __init__(
        self,
        train_df: pl.DataFrame,
    ):
        self.train_df = train_df

    def _map_col(self, column: str, cast: pl.DataType = None) -> dict:
        uniques = sorted(self.train_df.select(pl.col(column)).unique().to_series().to_list())
        mapping = {val: idx for idx, val in enumerate(uniques)}

        for attr in ("train_df",):
            df = getattr(self, attr)
            df = df.with_columns(
                pl.col(column)
                .replace(mapping)
                .alias(column)
            )
            if cast is not None:
                df = df.with_columns(pl.col(column).cast(cast))
            setattr(self, attr, df)

        return mapping

    def run(self):
        self.train_df = self.train_df.with_columns(
            pl.col("source_type").fill_null("").alias("source_type")
        )

        self.mapping_product_ids = self._map_col("product_id")
        self.mapping_user_ids = self._map_col("user_id")
        self.mapping_source_types = self._map_col("source_type", cast=pl.Int8)

        self.train_df = self.train_df.with_columns(
            pl.col("action_type")
            .replace(self.mapping_action_types)
            .cast(pl.Int8)
            .alias("action_type")
        )

        self.targets = (
            self.train_df
            .filter(
                pl.col("request_id").is_not_null() &
                pl.col("action_type").is_in([0, 1]) &
                (pl.col("source_type") != self.mapping_source_types["ST_Catalog"])
            )
            .group_by([
                "user_id",
                "request_id",
                "product_id",
            ])
            .agg([
                pl.col("action_type").max(),
                pl.col("timestamp").min(),
                pl.col("source_type").mode().first()
            ])
        )

        requests_with_cartupdate_and_view = (
            self.targets
            .select(["request_id", "action_type", "timestamp"])
            .group_by("request_id")
            .agg([
                pl.col("action_type").max().alias("max_t"),
                pl.col("action_type").min().alias("min_t"),
                pl.len(),
                pl.col("timestamp").min().alias("req_ts")
            ])
            .with_columns(sum_targets=pl.col('max_t').add(pl.col('min_t')))
            .filter(pl.col('sum_targets') == 1)
            .filter(pl.col('len') >= 10)
            .select(["request_id", "req_ts"])
        )
        self.targets = (
            self.targets
            .drop("timestamp")
            .join(requests_with_cartupdate_and_view, on="request_id", how="inner")
            .with_columns(pl.col("req_ts").alias("timestamp"))
            .drop("req_ts")
        )
        self.targets = (
            self.targets
            .group_by(['user_id', 'request_id', 'timestamp', 'source_type'])
            .agg([
                pl.col('product_id'),
                pl.col('action_type'),
            ])
        )

        self.timesplit_valid_end = self.train_df["timestamp"].max()
        self.timesplit_valid_start = self.timesplit_valid_end - 30 * 24 * 60 * 60
        self.timesplit_train_end = self.timesplit_valid_start - 2 * 24 * 60 * 60
        self.timesplit_train_start = self.train_df["timestamp"].min()

        self.train_df = (
            self.train_df
            .filter(pl.col("action_type") != 0)
            .drop("request_id")
        )
        
        self.train_targets = self.targets.filter(
            pl.col("timestamp") <= self.timesplit_train_end
        )
        self.valid_targets = self.targets.filter(
            (pl.col("timestamp") > self.timesplit_valid_start) &
            (pl.col("timestamp") <= self.timesplit_valid_end)
        )
        self.train_history = self.train_df.filter(pl.col('timestamp') <= self.timesplit_train_end)
        self.valid_history = self.train_df.filter(pl.col('timestamp') > self.timesplit_train_end)

        return (
            self.train_history,
            self.valid_history,
            self.train_targets,
            self.valid_targets
        )

In [ ]:
preprocessor = Preprocessor(train_df)
train_history, valid_history, train_targets, valid_targets = preprocessor.run()

- train_history/valid_history - позитивные взаимодействия пользователей
- train_targets/valid_targets - группы для finetune

## 3. (1 балл) Подготовка данных для pretrain

Для pretrain и finetune работа происходит с двумя последовательностями: последовательностью позитивных взаимодействий и последовательностью request-ов пользователя. Далее будем называть их history и candidates. На pretrain нам будет нужна только history. 

#### Обрезание историй

У пользователей может быть разное количество позитивных событий в истории. Для простоты будет рассматривать последние 512 событий. Если вдруг их будет больше, то будем обрезать.

#### Схемы таблиц и пример

Схема для history имеет вид: 
```python
HISTORY_SCHEMA = pl.Struct({
    'source_type': pl.List(pl.Int64), 
    'action_type': pl.List(pl.Int64),
    'product_id': pl.List(pl.Int64),
    'position': pl.List(pl.Int64),
    'targets_inds': pl.List(pl.Int64),
    'targets_lengths': pl.List(pl.Int64), # количество таргет событий в истории
    'lengths': pl.List(pl.Int64), # длина всей истории 
}).
```
`position` это индексы событий в истории (нужно будут далее для позиционных эмбеддингов), `targets_inds` - индексы тех позиций, которые будут участвовать в подсчете функции потерь. Они нужны, чтобы разделять потерю по событиям из обучающий и валидационной частей. `targets_lengths` - количество таких событий в истории. 
Пример: Пусть есть некоторый пользователь с историей из позитивных взаимодействий длины 5. Пусть первые 3 события попадают в обучение, а последние 2 в валидацию. Тогда:
```python
history_train_sample = pl.DataFrame({
    'source_type': [1, 1, 2, 3, 4],
    'action_type': [1, 0, 1, 0, 1],
    'product_id': [1, 2, 3, 4, 5],
    'position': [0, 1, 2, 3, 4],
    'targets_inds': [0, 1, 2],
    'targets_lengths': [3],
    'lengths': [5]
})
history_valid_sample = pl.DataFrame({
    'source_type': [1, 1, 2, 3, 4],
    'action_type': [1, 0, 1, 0, 1],
    'product_id': [1, 2, 3, 4, 5],
    'position': [0, 1, 2, 3, 4],
    'targets_inds': [3, 4],
    'targets_lengths': [2],
    'lengths': [5]
})
```

In [ ]:
def ensure_sorted_by_timestamp(group: Iterable[Dict[str, Any]]) -> Generator[Dict[str, Any], None, None]:
    """
    Ensures that the given iterable of events is sorted by the 'timestamp' field.

    This function iterates over each event in the provided iterable and checks if the
    'timestamp' of the current event is greater than or equal to the 'timestamp' of the
    previous event. If any event has a 'timestamp' that is less than the previous event's
    'timestamp', an AssertionError is raised.

    @param group: An iterable of dictionaries, where each dictionary represents an event with at least a 'timestamp' key.
    @return: A generator yielding each event from the input iterable in order, ensuring they are sorted by 'timestamp'.
    @raises AssertionError: If the events are not sorted by 'timestamp'.
    """
    # TODO: your code here

In [ ]:
class Mapper(ABC):
    HISTORY_SCHEMA = pl.Struct({
        'source_type': pl.List(pl.Int64), 
        'action_type': pl.List(pl.Int64),
        'product_id': pl.List(pl.Int64),
        'position': pl.List(pl.Int64),
        'targets_inds': pl.List(pl.Int64),
        'targets_lengths': pl.List(pl.Int64), # количество таргет событий в истории
        'lengths': pl.List(pl.Int64), # длина всей истории 
    })
    CANDIDATES_SCHEMA = pl.Struct({
        'source_type': pl.List(pl.Int64), 
        'action_type': pl.List(pl.Int64),
        'product_id': pl.List(pl.Int64),
        'lengths': pl.List(pl.Int64), # длина каждого реквеста
        'num_requests': pl.List(pl.Int64) # общее количество реквестов у этого пользователя
    })

    def __init__(self, min_length: int, max_length: int):
        self._min_length: int = min_length
        self._max_length: int = max_length

    @abstractmethod
    def __call__(self, group: pl.DataFrame) -> pl.DataFrame:
        pass

    def get_empty_frame(self, candidates=False):
        return pl.DataFrame(schema=pl.Schema({
            'history': self.HISTORY_SCHEMA,
            **({'candidates': self.CANDIDATES_SCHEMA} if candidates else {})
        }))

Реализуйте структуру, которая будет:
- накапливать history для пользователя и оставлять последние `max_length`
- уметь обращаться по индексу к событию истории
- имеет метод `get(self, targets_ids)`, который будет превращать `self._data` в dict в соотвествии со схемой `HISTORY_SCHEMA` (см. history_train_sample, history_valid_sample).

In [ ]:
class HistoryDeque:
    def __init__(self, max_length):
        self._data = deque([], maxlen=max_length)
    
    def append(self, x):
        # TODO: your code here
    
    def __len__(self):
        # TODO: your code here
    
    def __getitem__(self, idx):
        # TODO: your code here
    
    def get(self, targets_inds=None):
        """
        Retrieves a dictionary containing various attributes of the dataset samples.

        If `targets_inds` is not provided, it automatically identifies indices of samples where the `target` is 1.

        @param targets_inds: List of indices of target samples. If None, it will be determined based on samples with target value 1.
        @return: Dictionary with keys ['source_type', 'action_type', 'product_id', 'position', 'targets_inds', 'targets_lengths', 'lengths']
                Each key maps to a list or value representing the respective attribute of the dataset samples.
        """
        # TODO: your code here

На основе функции `get_pretrain_data` реализуйте `PretrainMapper`, который будет по данном пользователю выдавать обучающий пример в нужном формате. Используйте `HistoryDeque` и `get_empty_frame`.

In [ ]:
class PretrainMapper(Mapper):
    def __call__(self, group: pl.DataFrame) -> pl.DataFrame:
        """
        Processes a group of data by maintaining a history of rows up to a specified maximum length.
        If the history meets the minimum length requirement and contains at least one target, it returns
        a DataFrame with the history. Otherwise, it returns an empty DataFrame.

        @param group: A Polars DataFrame containing the group of data to process.
        @return: A Polars DataFrame containing the history if conditions are met; otherwise, an empty DataFrame.
        """
        # TODO: your code here

In [ ]:
def get_pretrain_data(train_history: pl.DataFrame,
                      valid_history: pl.DataFrame,
                      min_length: int = 5,
                      max_length: int = 4096) -> pl.DataFrame:
    mapper = PretrainMapper(
        min_length=min_length,
        max_length=max_length,
    )

    train_data = (
        train_history.with_columns(target=pl.lit(1)) 
        .sort(['user_id', 'timestamp'])
        .group_by('user_id')
        .map_groups(mapper)
    )

    valid_data = (
        pl.concat([
            train_history.with_columns(target=pl.lit(0)), 
            valid_history.with_columns(target=pl.lit(1))
        ], how='diagonal')
        .sort(['user_id', 'timestamp'])
        .group_by('user_id')
        .map_groups(mapper)
    )

    return train_data, valid_data 

In [ ]:
pretrain_train_data, pretrain_valid_data = get_pretrain_data(train_history, valid_history, min_length=5, max_length=512)

## 4. (1 балл) Реализуем свой torch.nn.utils.data.Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import polars as pl
from tqdm import tqdm

In [ ]:
def convert_dict_to_tensor(data_dict):
    """
    Recursively converts lists within a dictionary to PyTorch tensors with dtype=torch.int64.
    
    @param data_dict: A dictionary potentially containing nested dictionaries and lists.
    @return: A new dictionary with the same structure as `data_dict`, but with lists converted to PyTorch tensors.
    """
    # TODO: your code here


In [ ]:
def test_convert_dict_to_tensor_basic():
    input_data = {
        'a': [1, 2, 3],
        'b': {
            'c': [4, 5],
            'd': 6
        },
        'e': 'text'
    }

    result = convert_dict_to_tensor(input_data)

    assert isinstance(result['a'], torch.Tensor)
    assert result['a'].dtype == torch.int64
    assert result['a'].tolist() == [1, 2, 3]

    assert isinstance(result['b'], dict)
    assert isinstance(result['b']['c'], torch.Tensor)
    assert result['b']['c'].dtype == torch.int64
    assert result['b']['c'].tolist() == [4, 5]

    assert result['b']['d'] == 6
    assert result['e'] == 'text'

test_convert_dict_to_tensor_basic()

In [ ]:
class LavkaDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    @classmethod
    def from_dataframe(cls, df: pl.DataFrame) -> 'LavkaDataset':
        # TODO: your code here, use convert_dict_to_tensor
    
    def __len__(self):
        # TODO: your code here
    
    def __getitem__(self, idx):
        # TODO: your code here

In [ ]:
print("Creating train dataset ...")
train_ds = LavkaDataset.from_dataframe(pretrain_train_data)
print("Creating valid dataset ...")
valid_ds = LavkaDataset.from_dataframe(pretrain_valid_data)

## 5. (2 балл) Реализуем основной backbone модели

**Нигде далее при реализации моделей циклы использовать нельзя. Структуру кода везде можно менять без нарушения логики.**

Реализуйте класс ResNet-блока согласно следующим формулам:

Для входа $x\in\mathbb{R}^{\text{batch}\times d}$ вычислить:  
$$
    \begin{aligned}
    z &= xW + b,\\
    a &= \mathrm{ReLU}(z),\\
    d'&= \mathrm{Dropout}(a),\\
    y &= \mathrm{LayerNorm}\bigl(x + d'\bigr).
    \end{aligned}
$$  
В компактном виде:  
$$
    y = \mathrm{LayerNorm}\Bigl(x + \mathrm{Dropout}\bigl(\mathrm{ReLU}(xW + b)\bigr)\Bigr)\,.
$$


In [ ]:
class ResNet(nn.Module):
    def __init__(self, embedding_dim, dropout=0.):
        super().__init__()
        # TODO: your code here

    def forward(self, x):
        # TODO: your code here

In [ ]:
def test_resnet_output_shape():
    embedding_dim = 32
    batch_size = 8
    model = ResNet(embedding_dim)
    x = torch.randn(batch_size, embedding_dim)
    out = model(x)
    assert out.shape == (batch_size, embedding_dim)
    
test_resnet_output_shape()

Реализуйте `ContextEncoder`, `ItemEncoder` и `ActionEncoder`. На вход они принмают torch.tensor с индексами размера (seq_len,), а на выходе должны получить torch.tensor с соотвествующими векторами размера (seq_len, embedding_dim). Для `nn.Embedding` нужно будет подсчитать количество уникальных индексов для каждого типа. 

In [ ]:
class ContextEncoder(nn.Module):
    def __init__(self, embedding_dim=64):
        super().__init__()
        # TODO: your code here

    def forward(self, inputs):
        # TODO: your code here


class ItemEncoder(nn.Module):
    def __init__(self, embedding_dim=64):
        super().__init__()
        # TODO: your code here

    def forward(self, inputs):
        # TODO: your code here


class ActionEncoder(nn.Module):
    def __init__(self, embedding_dim=64):
        super().__init__()
        # TODO: your code here
    
    def forward(self, inputs):
        # TODO: your code here

In [ ]:
def test_context_encoder_shape():
    embedding_dim = 16
    batch_size = 5
    seq_len = 7
    model = ContextEncoder(embedding_dim=embedding_dim)
    x = torch.randint(0, 4, (batch_size, seq_len))
    out = model(x)
    assert out.shape == (batch_size, seq_len, embedding_dim)
test_context_encoder_shape()

def test_item_encoder_shape():
    embedding_dim = 20
    batch_size = 6
    seq_len = 10
    model = ItemEncoder(embedding_dim=embedding_dim)
    x = torch.randint(0, 20000, (batch_size, seq_len))
    out = model(x)
    assert out.shape == (batch_size, seq_len, embedding_dim)
test_item_encoder_shape()

def test_action_encoder_shape():
    embedding_dim = 12
    batch_size = 4
    seq_len = 3
    model = ActionEncoder(embedding_dim=embedding_dim)
    x = torch.randint(0, 4, (batch_size, seq_len))
    out = model(x)
    assert out.shape == (batch_size, seq_len, embedding_dim)
test_action_encoder_shape()

In [ ]:
def get_mask(lengths):
    """
    Generates a mask tensor based on the given sequence lengths.

    The mask is a boolean tensor where each row corresponds to a sequence and contains
    True values up to the length of the sequence and False values thereafter.

    @param lengths: A 1D tensor containing the lengths of sequences.
    @return: A 2D boolean tensor where each row has True up to the corresponding sequence length.
    """
    # TODO: your code here

In [ ]:
def test_get_mask():
    lengths = torch.tensor([2, 3, 1])
    expected_mask = torch.tensor([
        [True, True, False],
        [True, True, True],
        [True, False, False]
    ])
    assert torch.equal(get_mask(lengths), expected_mask)

test_get_mask()

Далее нужно реализовать `ModelBackbone`. Эта часть модели является общей для pretrain и finetune. Она кодируется входные события, преобразует их в нужный для трансформера формат `(batch_size, seq_len, embedding_dim)`, прогоняет через них трансформер и возвращает три поля: выходы трансформера, вектора товаров и вектора feedback-ов.

Трансформер применяем с каузальной маской. Не забудьте про кодирование позиций. 

In [ ]:
class ModelBackbone(nn.Module):
    def __init__(self, 
                 embedding_dim=64,
                 num_heads=2,
                 max_seq_len=512,
                 dropout_rate=0.2,
                 num_transformer_layers=2):
        super().__init__()
        self.context_encoder = # TODO: your code here
        self.item_encoder = # TODO: your code here
        self.action_encoder = # TODO: your code here
        self.position_embeddings = # TODO: your code here
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=embedding_dim * 4,
            dropout=dropout_rate,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_transformer_layers)
        self._embedding_dim = embedding_dim
    
    @property
    def embedding_dim(self):
        return self._embedding_dim
        
    def forward(self, inputs):
        context_embeddings = # TODO: your code here
        item_embeddings = # TODO: your code here
        action_embeddings = # TODO: your code here

        padding_mask = # TODO: your code here
        batch_size, seq_len = padding_mask.shape

        token_embeddings = item_embeddings.new_zeros(
            batch_size, seq_len, self.embedding_dim
        )
        token_embeddings[padding_mask] = # TODO: your code here

        # apply transformer encoder
        source_embeddings = # TODO: your code here

        return {
            'source_embeddings': source_embeddings,
            'item_embeddings': item_embeddings,
            'context_embeddings': context_embeddings
        }

In [ ]:
def test_model_backbone():
    sample = {
        'history': {
            'source_type': torch.tensor([1, 1, 7, 1, 1]), 
            'action_type': torch.tensor([2, 2, 2, 1, 2]), 
            'product_id': torch.tensor([19210,  8368,  5165,  5326, 12476]), 
            'position': torch.tensor([0, 1, 2, 3, 4]), 
            'targets_inds': torch.tensor([0, 1, 2]), 
            'targets_lengths': torch.tensor([3]), 
            'lengths': torch.tensor([5])
        }
    }
    backbone = ModelBackbone()
    output = backbone(sample)
    assert output['source_embeddings'].shape == (5, 64)
    assert output['item_embeddings'].shape == (5, 64)
    assert output['context_embeddings'].shape == (5, 64)
test_model_backbone()

## 6. (2 балл) Реализуем pretrain модель

#### Головы (heads)

- **User–context fusion**  
      Последовательность ResNet-блоков, свёртка размерности `2D → D`. На входе конкатенация `source_embeddings ∥ context_embeddings`.  
- **Candidate projector**  
      Три ResNet-блока, преобразующие эмбеддинг товара из `D → D`.  
- **Classifier**  
      Три ResNet-блока, затем линейный слой `3D → 3`, для предсказания типа следующего действия из трёх возможных (cart, click, purchase).  
- **Параметр τ**  
      Скаляp-коэффициент `τ = clip(exp(τ_raw), τ_min, τ_max)` для масштабирования скалярных произведений – температура в contrastive-лоссе.


#### Формулы лоссов

Обозначения:  
- $u_i \in \mathbb{R}^D$ – нормализованный вектор пользователя для i-го примера.  
- $c_i \in \mathbb{R}^D$ – нормализованный вектор кандидата (позитивного товара) для i-го примера.  
- $\{n_{ij}\}_{j=1}^M\subset\mathbb{R}^D$ – нормализованные векторы $M$ негативных товаров (весь каталог товаров).  
- $\tau>0$ – «температура» (скаляр).  
- $K= M+1$ – общее число кандидатов (1 позитивный + M негативных).

1. **Retrieval loss** (контрастивный softmax-лосс)  
   Для каждого примера $i$ вычисляем скалярные логиты:  
   $$
     \ell_i = [\,\underbrace{u_i^\top c_i}_{\text{позитивный логит}} \,\big|\,
               \underbrace{u_i^\top n_{i1},\,u_i^\top n_{i2},\,\dots,\,u_i^\top n_{iM}}_{\text{негативные логиты}}] \;\times\;\tau
   $$
   Затем лосс  
   $$
     \mathcal{L}_{\mathrm{retr}} 
     = -\frac1N \sum_{i=1}^N \log\frac{\exp\bigl(u_i^\top c_i \,\tau\bigr)}
                                      {\exp\bigl(u_i^\top c_i \,\tau\bigr)
                                     + \sum_{j=1}^M \exp\bigl(u_i^\top n_{ij}\,\tau\bigr)}.
   $$

2. **Action loss** (кросс-энтропия)  
   Для каждого положительного шага $i$ модель выдаёт логиты $\mathbf{z}_i \in \mathbb{R}^3$ по трём классам действий, а истинная метка $y_i\in\{0,1,2\}$.  
   $$
     \mathcal{L}_{\mathrm{action}} 
     = -\frac1N \sum_{i=1}^N \sum_{k=0}^2 \delta_{y_i=k}\,\log\bigl(\mathrm{softmax}(\mathbf{z}_i)_k\bigr),
   $$
   где $\mathrm{softmax}(\mathbf{z})_k = \frac{e^{z_k}}{\sum_{m=0}^2 e^{z_m}}$.

3. **Итоговый лосс**  
   $$
     \mathcal{L} 
     = \mathcal{L}_{\mathrm{retr}} 
       \;+\; 10 \times \mathcal{L}_{\mathrm{action}}.
   $$
   Перевзвешиваем action часть т.к. у нее сильно меньше масштаб.

#### Подсказки
Используйте `tensor.roll` и `tensor.log_softmax`, `torch.repeat_interleave`.

In [ ]:
class PretrainModel(nn.Module):
    MIN_TEMPERATURE = 0.01
    MAX_TEMPERATURE = 100
    
    def __init__(self,
                 backbone,
                 embedding_dim=64):
        super().__init__()
        self.backbone = backbone 
        self.user_context_fusion = nn.Sequential(
            ResNet(2 * embedding_dim),
            ResNet(2 * embedding_dim),
            ResNet(2 * embedding_dim),
            nn.Linear(2 * embedding_dim, embedding_dim),
        )
        self.candidate_projector = nn.Sequential(
            ResNet(embedding_dim),
            ResNet(embedding_dim),
            ResNet(embedding_dim),
        )
        self.classifier = nn.Sequential(
            ResNet(3 * embedding_dim),
            ResNet(3 * embedding_dim),
            ResNet(3 * embedding_dim),
            nn.Linear(3 * embedding_dim, 3),
        )
        self._embedding_dim = embedding_dim
        self.tau = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))
    
    @property
    def embedding_dim(self):
        return self._embedding_dim

    @property
    def temperature(self):
        return torch.clip(torch.exp(self.tau), min=self.MIN_TEMPERATURE, max=self.MAX_TEMPERATURE)
        
    def forward(self, inputs):
        backbone_outputs = # TODO: your code here
        source_embeddings = # TODO: your code here
        item_embeddings = # TODO: your code here
        context_embeddings = # TODO: your code here

        lengths = # TODO: your code here
        offsets = # TODO: your code here
        target_mask = # TODO: your code here
        target_inds = # TODO: your code here
        target_mask[target_inds] = # TODO: your code here

        non_first_element = # TODO: your code here
        non_first_element[offsets - lengths] = # TODO: your code here
        non_first_element &= # TODO: your code here
        source_mask = # TODO: your code here

        source_embeddings = source_embeddings[source_mask]
        context_embeddings = context_embeddings[non_first_element]
        item_embeddings = item_embeddings[non_first_element]
        
        # calc retrieval loss
        user_embeddings = torch.nn.functional.normalize(# TODO: your code here)
        candidate_embeddings = torch.nn.functional.normalize(# TODO: your code here)
        negative_embeddings = torch.nn.functional.normalize(# TODO: your code here)
        pos_logits = # TODO: your code here
        neg_logits = # TODO: your code here
        next_positive_prediction_loss = # TODO: your code here

        # calc action loss
        logits = # TODO: your code here 
        targets = # TODO: your code here
        feedback_prediction_loss = # TODO: your code here

        return {
            'next_positive_prediction_loss': next_positive_prediction_loss,
            'feedback_prediction_loss': feedback_prediction_loss,
            'loss': next_positive_prediction_loss + feedback_prediction_loss * 10
        }

## 7. (0.5 балл) Обучим pretrain модель

In [ ]:
def collate_fn(batch):
    """
    Collates a batch of samples from a dataset.

    This function is designed to handle batches where each sample is a dictionary.
    It recursively collates values associated with the same keys across all samples in the batch.
    For tensor values, it concatenates them along the first dimension.
    For dictionary values, it applies the same collation logic recursively.
    For other types of values, it simply aggregates them into a list.

    @param batch: A list of samples, where each sample is a dictionary.
    @return: A dictionary with the same keys as the samples, where each value is either
             a concatenated tensor, a recursively collated dictionary, or a list of values.
    """
    # TODO: your code here 


def move_to_device(batch, device):
    """
    Moves a batch of data to a specified device (e.g., CPU or GPU).

    Args:
        batch (torch.Tensor or dict): The batch of data to move. Can be a single tensor or a dictionary of tensors.
        device (torch.device): The target device to which the batch should be moved.

    Returns:
        torch.Tensor or dict: The batch of data moved to the specified device. 
                             If the input is a dictionary, the returned value will be a dictionary with the same keys 
                             and values moved to the specified device.
    """
    # TODO: your code here 

In [ ]:
def test_collate_fn_basic():
    batch = [
        {
            'x': torch.tensor([1, 2]),
            'y': {
                'z': torch.tensor([[10], [20]]),
                'w': 5
            },
            's': 'foo'
        },
        {
            'x': torch.tensor([3, 4]),
            'y': {
                'z': torch.tensor([[30], [40]]),
                'w': 6
            },
            's': 'bar'
        }
    ]

    result = collate_fn(batch)

    assert isinstance(result['x'], torch.Tensor)
    assert result['x'].tolist() == [1, 2, 3, 4]

    assert isinstance(result['y'], dict)
    assert isinstance(result['y']['z'], torch.Tensor)
    assert result['y']['z'].tolist() == [[10], [20], [30], [40]]
    assert result['y']['w'] == [5, 6]

    assert result['s'] == ['foo', 'bar']
test_collate_fn_basic()

In [ ]:
from statistics import mean
from sklearn.metrics import ndcg_score


def train_pretrain_model(model, train_loader, valid_loader, optimizer, scheduler, num_epochs, device):
    global_cnt = 0
    prev_valid_loss = None
    for epoch in range(num_epochs):
        model.train()
        train_losses = [] 
        action_losses = []
        retrieval_losses = []
        for batch in tqdm(train_loader):
            batch = move_to_device(batch, device)
            optimizer.zero_grad()
            output = model(batch)
            loss = output['loss']
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            action_losses.append(output['feedback_prediction_loss'].item())
            retrieval_losses.append(output['next_positive_prediction_loss'].item())
        scheduler.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {mean(train_losses):.6f}, Train Feedback Loss: {mean(action_losses):.6f}, Train NPP Loss: {mean(retrieval_losses):.6f}")

        model.eval()
        valid_losses = []
        action_losses = []
        retrieval_losses = []
        with torch.inference_mode():
            for batch in tqdm(valid_loader):
                batch = move_to_device(batch, device)
                output = model(batch)
                loss = output['loss']
                valid_losses.append(loss.item())
                action_losses.append(output['feedback_prediction_loss'].item())
                retrieval_losses.append(output['next_positive_prediction_loss'].item())

        avg_valid_loss = mean(valid_losses)
        print(f"Epoch {epoch+1}/{num_epochs}, Valid Loss: {avg_valid_loss:.6f}, Valid Feedback Loss: {mean(action_losses):.6f}, Valid NPP Loss: {mean(retrieval_losses):.6f}")
        
        if prev_valid_loss is None or prev_valid_loss > avg_valid_loss:
            global_cnt = 0
            prev_valid_loss = avg_valid_loss
            with torch.no_grad():
                torch.save(model, './pretrain.pt')
        else:
            global_cnt += 1
            if global_cnt == 10:
                break

Параметры и инициализацию можно менять для пробития порогов:

In [ ]:
lr = 0.001
batch_size = 4
warmup_epochs = 3
start_factor = 0.1
num_epochs = 100

embedding_dim = 64
num_heads = 2
max_seq_len = 512
dropout_rate = 0.1
num_transformer_layers = 2

In [ ]:
train_loader = # TODO: your code here 
valid_loader = # TODO: your code here 

In [ ]:
device = torch.device('cuda')

backbone = # TODO: your code here 
model_pretrain = # TODO: your code here 
optimizer = optim.AdamW(model_pretrain.parameters(), lr=lr, weight_decay=0.01)
scheduler = optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=start_factor,
    total_iters=warmup_epochs
)

In [ ]:
train_pretrain_model(model_pretrain, train_loader, valid_loader, optimizer, scheduler, num_epochs, device)

In [ ]:
def test_pretrain_model(valid_loader):
    test_model = torch.load('./pretrain.pt', weights_only=False)
    valid_losses = []
    with torch.inference_mode():
        for batch in tqdm(valid_loader):
            batch = move_to_device(batch, device)
            output = test_model(batch)
            loss = output['loss']
            valid_losses.append(loss.item())
    assert mean(valid_losses) < 11.7

test_pretrain_model(valid_loader)

## 8. (1 балл) Подготовка данных для finetune

Схемы для candidates:
```python
CANDIDATES_SCHEMA = pl.Struct({
    'source_type': pl.List(pl.Int64), 
    'action_type': pl.List(pl.Int64),
    'product_id': pl.List(pl.Int64),
    'lengths': pl.List(pl.Int64), # длина каждого реквеста
    'num_requests': pl.List(pl.Int64) # общее количество реквестов у этого пользователя
})
```

Пример семпла:

```python
finetune_train_sample = {
    'history': {...},
    'candidates': {
        'source_type': [1, 2, 3],
        'action_type': [1, 0, 1, 0, 1, 0, 1, 1, 1],
        'product_id': [10, 20, 30, 40, 50, 60, 70, 80, 90],
        'lengths': [3, 3, 3],
        'num_requests': 3
    }
}
```

Аналогично реализуйте структуру для candidates

In [ ]:
class Candidates:
    def __init__(self, max_requests_size):
        self._data = deque([], maxlen=max_requests_size) 
    
    def append(self, x):
        # TODO: your code here 
    
    def popleft(self):
        # TODO: your code here 

    def __getitem__(self, idx):
        # TODO: your code here 
    
    def __len__(self):
        # TODO: your code here 
    
    def get(self):
       """
        Aggregates data from the internal _data attribute into a structured dictionary format.

        This method constructs a dictionary with keys 'source_type', 'action_type', 'product_id', 'lengths', and 'num_requests'.
        - 'source_type' contains the source types from each sample.
        - 'action_type' contains all action types from each sample's action_type_list flattened into a single list.
        - 'product_id' contains all product IDs from each sample's product_id_list flattened into a single list.
        - 'lengths' contains the length of the product_id_list for each sample.
        - 'num_requests' contains the total number of samples.

        Returns:
            Dict[str, Any]: A dictionary with aggregated data.
        """ 
        # TODO: your code here 

При реализации `FinetuneTrainMapper` и `FinetuneValidMapper` отличие будет только в формировании targets_inds. В первом случае это может быть некоторая произвольная последовательность индексов, т.к. присутсвует случайность. Во втором случае это всегда будет последовательность из просто одного и того же последнего индекса.

`FinetuneTrainMapper` можно реализовать за O(N^2) от длины candidates.

In [ ]:
import random


class FinetuneTrainMapper(Mapper):
    def __call__(self, group: pl.DataFrame) -> pl.DataFrame:
        """
        Processes a group of interactions to generate history and candidate sets for recommendation.

        This method processes a DataFrame containing interaction data, separating actions into history and candidates based on the presence of 'action_type_list'.
        It ensures the data is sorted by timestamp, filters candidates based on time constraints, and selects historical interactions within a specified lag range for each candidate.
        If there are no valid candidates or insufficient history, it returns an empty DataFrame.

        @param group: A Polars DataFrame containing interaction data with at least 'timestamp' and 'action_type_list' columns.
        @return: A Polars DataFrame with 'history' and 'candidates' columns, or an empty DataFrame if no valid candidates are found.
        """
        # TODO: your code here 
        

class FinetuneValidMapper(Mapper):
    def __call__(self, group: pl.DataFrame) -> pl.DataFrame:
        """
        Differs only in the formation of target_inds
        """
        # TODO: your code here 

In [ ]:
def get_finetune_data(train_history: pl.DataFrame,
                      train_targets: pl.DataFrame,
                      valid_targets: pl.DataFrame,
                      min_length: int = 5,
                      max_length: int = 4096) -> pl.DataFrame:
    mapper = FinetuneTrainMapper(
        min_length=min_length,
        max_length=max_length,
    )

    train_data = (
        pl.concat([
            train_history,
            train_targets.with_columns([
                pl.col('product_id').alias('product_id_list'), 
                pl.col('action_type').alias('action_type_list')
            ]).drop(['product_id', 'action_type'])
        ], how='diagonal')
        .sort(['user_id', 'timestamp'])
        .group_by('user_id')
        .map_groups(mapper)
    )

    mapper = FinetuneValidMapper(
        min_length=min_length,
        max_length=max_length,
    )

    valid_data = (
        pl.concat([
            train_history,
            valid_targets.with_columns([
                pl.col('product_id').alias('product_id_list'), 
                pl.col('action_type').alias('action_type_list')
            ]).drop(['product_id', 'action_type'])
        ], how='diagonal')
        .sort(['user_id', 'timestamp'])
        .group_by('user_id')
        .map_groups(mapper)
    )

    return train_data, valid_data 

In [ ]:
finetune_train_data, finetune_valid_data = get_finetune_data(train_history, train_targets, valid_targets, min_length=5, max_length=512)

In [ ]:
print("Creating train dataset ...")
train_ds = LavkaDataset.from_dataframe(finetune_train_data)
print("Creating valid dataset ...")
valid_ds = LavkaDataset.from_dataframe(finetune_valid_data)

## 9. (2 балл) Реализуем finetune модель

#### Функция make_groups: разметка элементов по «группам»

Дано: вектор длин последовательностей  
$$
\mathbf{l} = [\,l_1, l_2, \dots, l_B\,],\quad l_i\in\mathbb{N},\;
B=\text{batch size}.
$$  
Нужно получить вектор «номеров групп» длиной  
$$
N = \sum_{i=1}^B l_i
$$
так, чтобы первые $l_1$ элементов имели номер группы 0, следующие $l_2$ - номер 1 и т.д.  
Результат:  
$$
\mathrm{groups} = [\,\underbrace{0,\dots,0}_{l_1},\;
\underbrace{1,\dots,1}_{l_2},\;\dots\;,\underbrace{B-1,\dots,B-1}_{l_B}\,]\,.
$$

Циклы использовать нельзя.

In [ ]:
def make_groups(lengths: torch.Tensor) -> torch.Tensor:
    # TODO: your code here 

In [ ]:
def test_make_groups_basic():
    lengths = torch.tensor([2, 3, 1])
    expected = torch.tensor([0, 0, 1, 1, 1, 2])
    result = make_groups(lengths)
    assert torch.equal(result, expected)
    
test_make_groups_basic()

#### Функция make_pairs: построение всех упорядоченных пар внутри групп

Цель: для каждого «блока» длины $l_i$ сгенерировать всех $l_i\times l_i$ упорядоченных пар индексов  
$$
( p, q ),\quad p,q\in\{0,\dots,l_i-1\},
$$
а затем «развернуть» их по всему батчу. Результат - двумерный тензор shape $(2,\,\sum_i l_i^2)$, где

- первая строка `pairs` - индексы «первого» элемента пары в пределах своего блока,  
- вторая строка `pairs` - индексы «второго».

Математически пары нумеруются так:
$$
\{\, (p,q)\;\big|\;p=0..l_i-1,\;q=0..l_i-1\;\}\quad\forall i=1..B.
$$
Используйте scatter_add:  
- один - чтобы повторить номер группы и получить `first`,  
- другой - чтобы вычислить смещение для `second`.

In [ ]:
def make_pairs(lengths: torch.Tensor) -> torch.Tensor:
    # TODO: your code here 

In [ ]:
def test_make_pairs_simple():
    lengths = torch.tensor([1, 2], dtype=torch.long)
    expected = torch.tensor([
        [0, 1, 1, 2, 2],
        [0, 1, 2, 1, 2]
    ], dtype=torch.long)

    pairs = make_pairs(lengths)
    assert pairs.shape == (2, 5)
    assert torch.equal(pairs, expected)

test_make_pairs_simple()

#### Класс CalibratedPairwiseLogistic: попарная калиброванная логистическая функция потерь

Идея была предложена здесь: [Calibrated Pairwise Logistic](https://arxiv.org/pdf/2211.01494). Пусть у нас есть:

- логиты всех элементов: $\mathbf{c} \in \mathbb{R}^N$,
- таргеты $\mathbf{t}\in\mathbb{R}^N$,

Шаги:

1. Генерируем все упорядоченные пары индексов внутри групп:  
   $$
   \mathrm{pairs} = \bigl[\;I_0,\;I_1\bigr],\quad
   I_0,I_1\in\{0,\dots,N-1\}
   $$
2. Для каждой пары извлекаем  
   $$
   c_i = c_{I_0},\quad c_j = c_{I_1},\quad
   t_i = t_{I_0},\quad t_j = t_{I_1}.
   $$
3. Отбираем только «положительные» пары, где $t_i > t_j$. Вводим индикатор  
   $$
   w_{ij} = 
     \begin{cases}
       1,&t_i > t_j,\\
       0,&\text{иначе}.
     \end{cases}
   $$
   И считаем $W=\sum w_{ij}$.
4. Если $W>0$, вычисляем попарный loss для каждой положительной пары:
   
   а) сначала вычисляем «калиброванную вероятность» того, что $i$ лучше $j$:
   $$
     p_{ij}
     = \frac{\sigma(c_i)}{\sigma(c_i)+\sigma(c_j)},
     \quad
     \sigma(x)=\frac1{1+e^{-x}}.
   $$
   б) берём отрицательный логарифм правдоподобия:
   $$
     \ell_{ij}
     = -\log p_{ij}
     = -\log\frac{\sigma(c_i)}{\sigma(c_i)+\sigma(c_j)}.
   $$
   
5. Итоговая loss - усреднённая:
   $$
     \mathcal{L}
     = \frac{1}{W}\sum_{i,j} w_{ij}\;\ell_{ij}.
   $$
6. Если $W=0$ (нет ни одной пары с $t_i>t_j$), возвращаем нуль.

Таким образом, CalibratedPairwiseLogistic минимизирует  
$$
-\frac{1}{W}\sum_{t_i>t_j}\log\frac{\sigma(c_i)}{\sigma(c_i)+\sigma(c_j)},
$$
то есть учит давать более высокие оценки $c_i$ элементам с большим таргетом $t_i$.

Подсказка: используйте `make_pairs`, `log_softmax`, `logsigmoid`. 

In [ ]:
class CalibratedPairwiseLogistic(nn.Module):
    def forward(self, logits, targets, lengths):
        # TODO: your code here 

В FinetuneModel к сырым логитам  
$$\ell_i = \langle u_i, v_i\rangle$$  
применяется калибровка:

1. Параметр «scale» (обозначим $s$) хранится в виде логарифма, то есть в модели он задан как $\text{scale}$, а реальный множитель берётся как  
   $$
     \alpha = \exp(\text{scale}).
   $$

2. Параметр «bias» (обозначим $b$) - это свободный смещающий коэффициент.

Калиброванный логит получается по формуле  
$$
  \hat\ell_i \;=\; \frac{\ell_i}{\alpha} \;+\; b
  \;=\;
  \frac{\langle u_i, v_i\rangle}{\exp(\text{s})} \;+\; b.
$$

Благодаря этому механизмy модель может автоматически подстраивать и жёсткость (разброс) логитов (через $\alpha$), и их среднее значение (через $b$), что важно для оптимальной работы попарной логистической функции потерь.

In [ ]:
class FinetuneModel(nn.Module):
    def __init__(self,
                 backbone,
                 embedding_dim=64):
        super().__init__()
        self.backbone = backbone
        self.user_context_fusion = nn.Sequential(
            ResNet(2 * embedding_dim),
            ResNet(2 * embedding_dim),
            ResNet(2 * embedding_dim),
            nn.Linear(2 * embedding_dim, embedding_dim),
        )
        self.candidate_projector = nn.Sequential(
            ResNet(embedding_dim),
            ResNet(embedding_dim),
            ResNet(embedding_dim),
        )
        self._embedding_dim = embedding_dim
        self.scale = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))
        self.bias = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))
        self.pairwise_loss = CalibratedPairwiseLogistic()
    
    @property
    def embedding_dim(self):
        return self._embedding_dim
        
    def forward(self, inputs):
        backbone_outputs = # TODO: your code here 
        source_embeddings = # TODO: your code here 

        lengths = # TODO: your code here 
        offsets = # TODO: your code here 
        target_inds = # TODO: your code here 
        source_embeddings = # TODO: your code here 

        source_embeddings = torch.nn.functional.normalize(# TODO: your code here)
        candidate_embeddings = torch.nn.functional.normalize(# TODO: your code here )
        source_embeddings = # TODO: your code here 
        output_logits = # TODO: your code here 

        return {
            'logits': output_logits,
            'loss': 
        }

In [ ]:
def test_finetune_model():
    sample = {
        'history': {
            'source_type': torch.tensor([8, 8, 8, 8, 8]), 
            'action_type': torch.tensor([1, 1, 2, 2, 1]), 
            'product_id': torch.tensor([ 3551, 17044, 10396, 10396, 10396]), 
            'position': torch.tensor([0, 1, 2, 3, 4]), 'targets_inds': torch.tensor([1]), 
            'targets_lengths': torch.tensor([1]), 
            'lengths': torch.tensor([5])
        }, 
        'candidates': {
            'source_type': torch.tensor([8]), 
            'action_type': torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), 
            'product_id': torch.tensor([18391,  6750, 21647,  5339,  3171,  6150,  3454, 20012, 19954, 10690, 24020,  5551,  5699, 17388, 10396]), 
        'lengths': torch.tensor([15]), 
        'num_requests': torch.tensor([1])
        }
    }
    backbone = ModelBackbone()
    model_finetune = FinetuneModel(backbone)
    output = model_finetune(sample)
    assert output['logits'].shape == (15,)
    assert output['loss'].shape == ()

test_finetune_model()

## 10. (0.5 балл) Обучаем finetune модель

In [ ]:
def train_finetune_model(model, train_loader, valid_loader, optimizer, scheduler, num_epochs, device):
    prev_valid_ndcg = None
    global_cnt = 0
    for epoch in range(num_epochs):
        model.train()
        train_losses = [] 
        for batch in tqdm(train_loader):
            batch = move_to_device(batch, device)
            optimizer.zero_grad()
            loss = model(batch)['loss']
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        scheduler.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {mean(train_losses):.6f}")

        model.eval()
        valid_losses = []
        valid_logits = [] 
        valid_targets = []
        with torch.inference_mode():
            for batch in tqdm(valid_loader):
                batch = move_to_device(batch, device)
                output = model(batch)
                loss = output['loss']
                valid_losses.append(loss.item())
                logits = output['logits']
                targets = batch['candidates']['action_type']
                lengths = batch['candidates']['lengths']
                i = 0
                for length in lengths:
                    if length > 1:
                        valid_logits.append(logits[i:i + length].cpu().numpy())
                        valid_targets.append(targets[i:i + length].cpu().numpy())
                    i += length

        avg_valid_ndcg = 0
        for logits, targets in zip(valid_logits, valid_targets):
            avg_valid_ndcg += ndcg_score(targets[None,], logits[None,], k=10, ignore_ties=True)
        avg_valid_ndcg /= len(valid_logits)

        print(f"Epoch {epoch+1}/{num_epochs}, Valid Loss: {mean(valid_losses):.6f}")
        print(f"Epoch {epoch+1}/{num_epochs}, Valid NDCG@10: {avg_valid_ndcg}")
        
        if prev_valid_ndcg is None or prev_valid_ndcg < avg_valid_ndcg:
            global_cnt = 0
            prev_valid_ndcg = avg_valid_ndcg
            with torch.no_grad():
                torch.save(model, './finetune.pt')
        else:
            global_cnt += 1
            if global_cnt == 10:
                break

Параметры и инициализации можно менять для пробития порогов:

In [ ]:
lr = 0.001
batch_size = 4 
warmup_epochs = 3
start_factor = 0.1
num_epochs = 100

embedding_dim = 64
num_heads = 2
max_seq_len = 512
dropout_rate = 0.1
num_transformer_layers = 2

In [ ]:
train_loader = # TODO: your code here 
valid_loader = # TODO: your code here 

In [ ]:
backbone = # TODO: your code here 
model_finetune = # TODO: your code here 
optimizer = optim.AdamW(model_finetune.parameters(), lr=lr, weight_decay=0.01)
scheduler = optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=start_factor,
    total_iters=warmup_epochs
)

In [ ]:
train_finetune_model(model_finetune, train_loader, valid_loader, optimizer, scheduler, num_epochs, device)

Пробуем инициализироваться предобученной моделью, только backbone:

In [ ]:
model_pretrain = torch.load('./pretrain.pt', weights_only=False)
model_finetune = FinetuneModel(model_pretrain.backbone, embedding_dim).to(device)
optimizer = optim.AdamW(model_finetune.parameters(), lr=lr, weight_decay=0.01)
scheduler = optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=start_factor,
    total_iters=warmup_epochs
)

In [ ]:
train_finetune_model(model_finetune, train_loader, valid_loader, optimizer, scheduler, num_epochs, device)

Попробуем еще дополнительно иницилизировать user_context_fusion и candidate_projector:

In [ ]:
model_pretrain = torch.load('./pretrain.pt', weights_only=False)
model_finetune = FinetuneModel(model_pretrain.backbone, embedding_dim).to(device)
model_finetune.user_context_fusion = model_pretrain.user_context_fusion
model_finetune.candidate_projector = model_pretrain.candidate_projector

optimizer = optim.AdamW(model_finetune.parameters(), lr=lr, weight_decay=0.01)
scheduler = optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=start_factor,
    total_iters=warmup_epochs
)

In [ ]:
train_finetune_model(model_finetune, train_loader, valid_loader, optimizer, scheduler, num_epochs, device)

In [ ]:
def test_finetune_model(valid_loader):
    valid_logits = []
    valid_targets = []
    with torch.inference_mode():
        test_model = torch.load('./finetune.pt', weights_only=False)
        for batch in tqdm(valid_loader):
            batch = move_to_device(batch, device)
            output = test_model(batch)
            logits = output['logits']
            targets = batch['candidates']['action_type']
            lengths = batch['candidates']['lengths']
            i = 0
            for length in lengths:
                if length > 1:
                    valid_logits.append(logits[i:i + length].cpu().numpy())
                    valid_targets.append(targets[i:i + length].cpu().numpy())
                i += length

    avg_valid_ndcg = 0
    for logits, targets in zip(valid_logits, valid_targets):
        avg_valid_ndcg += ndcg_score(targets[None,], logits[None,], k=10, ignore_ties=True)
    avg_valid_ndcg /= len(valid_logits)
    assert avg_valid_ndcg > 0.324

test_finetune_model(valid_loader)

#### Вывод
- Напишите вывод про влияние pretrain на качество решаемой задачи.

...

## 11. Оцениваем качество в CatBoost и пробуем в контесте (этот пункт не оценивается)

- Аналогично тому, как в домашнtv задании мы получали логиты для валидации, можно сгенерировать логиты и на тестовом наборе из контеста. После этого получившийся скаляр присоединяется к фичам CatBoost для трейна и теста:  
    ```python
    train_catboost = train_catboost.join(
        argus_score,
        on=['request_id', 'product_id'],
        how='left'
    )
    valid_catboost = valid_catboost.join(
        argus_score,
        on=['request_id', 'product_id'],
        how='left'
    )
    ```
  Джойн выполняется по паре `(request_id, product_id)`. Чтобы это работало, в `Mapper`, `HistoryDeque` и `Candidates` нужно дополнительно передавать поле `request_id` в виде строки.

- **Очень важно!** Периоды обучения трансформера и CatBoost не должны пересекаться. Иначе через этот признак «просочится» таргет, и CatBoost переобучится: вы увидите его на первой позиции по fstr, но реального улучшения в контесте не будет.

<div align="center">
  <img src="https://i.ibb.co/mVcH51gW/IMG000-18.jpg" width="500" alt="as_feature">
</div>

- После джойна и обучения CatBoost признак окажется в числе первых по важности (fstr), вы получите прирост валидационных метрик и результатов на лидерборде Kaggle. Однако не стоит рассчитывать на космический эффект - обычно это +0.001…0.005, в зависимости от вышего набора признаков.

- На практике такие алгоритмы работают гораздо эффективнее, поскольку для нейросетевых рекомендательных алгоритмов критично большое количество данных. В контесте же мы оперируем очень скромным объёмом, и методы не могут раскрыться полностью.

## 12. (1 балл) Бонус 1: Расширение модели новыми признаками и текстовой модальностью

- Нейросети выигрывают у градиентного бустинга благодаря способности принимать на вход данные разных типов и структур: тексты, графы, изображения, аудио и т. д. В этом бонусе предлагается дополнить модель двумя категориальными признаками (`city_name`, `store_id`), двумя текстовыми (`product_name`, `product_category`) и закодировать `timestamp`.
- Для кодирования текстов используйте подход Bag of Words:  
  1) Токенизируйте `product_name` и `product_category` с помощью модели [DmitryPogrebnoy/distilbert-base-russian-cased](DmitryPogrebnoy/distilbert-base-russian-cased).  
  2) Получайте вектор признака как сумму эмбеддингов всех токенов.
- Для кодирования времени отдельно заведите категориальный признак для часа дня (24 уникальных значения), для дня недели (7 уникальных значений), для месяца (12 уникальных значений). Можете рассмотреть и более продвинутые схемы (см. appendix [Actions Speak Louder than Words: Trillion-Parameter Sequential Transducers for Generative Recommendations](https://arxiv.org/abs/2402.17152)). 
- Для объединения (агрегации) всех признаков в единый вектор внедрите CrossLayer из семинара 5. Для этого расширьте `ItemEncoder`, добавив в него Cross-слой, связывающий категориальные и текстовые фичи.
- Используйте одну и ту же матрицу эмбеддингов для текстовых признаков `product_name` и `product_category`.

In [ ]:
class CrossLayer(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # TODO: your code here

    def forward(self, x):
        # TODO: your code here

## 13. (1 балл) Бонус 2: Добавление в модель изображений

- Аналогично расширьте модель визуальной информацией.
- Учтите, что скачивание изображений может занять значительное время - до 8–10 часов.
- Для кодирования изображений в вектор пользуйтесь любой предобученной моделью.
- Пример скачивания картинок и сохранения их векторов:

    ```python
    import polars as pl
    import requests
    from PIL import Image
    from io import BytesIO
    import os
    import torch
    import torch.nn as nn
    import torchvision.models as models
    import torchvision.transforms as transforms
    from tqdm import tqdm
    import numpy as np

    PARQUET_PATH = 'product2image.parquet'
    EMBEDDING_SAVE_PATH = 'product_embeddings.pth'
    MAX_PRODUCT_ID = 26_110
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    df = pl.read_parquet(PARQUET_PATH)

    def download_and_process_image(url):
        """Downloads an image from URL, processes it, returns PIL Image or None on error."""
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(url, timeout=15, headers=headers, stream=True)
            response.raise_for_status()

            content_type = response.headers.get('Content-Type')
            if content_type and not content_type.startswith('image/'):
                print(f"Warning: URL does not point to an image (Content-Type: {content_type}). Skipping.")
                return None

            img = Image.open(BytesIO(response.content))
            if img.mode != 'RGB':
                img = img.convert('RGB')
            return img
        except requests.exceptions.RequestException as e:
            print(f"Warning: Network error downloading image from {url}: {e}")
            return None
        except Exception as e:
            print(f"Warning: Failed to process image from {url}: {e}")
            return None

    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    model = nn.Sequential(*list(model.children())[:-1])
    model.eval()
    model.to(DEVICE)

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    print("Extracting features from images...")
    product_vectors = {}
    embedding_dim = None
    processed_count = 0

    for row in tqdm(df.iter_rows(named=True), total=len(df), desc="Processing images"):
        product_id = row['product_id']
        url = row['product_image']

        if not (url and isinstance(url, str) and url.startswith('http')):
            print(f"Warning: Invalid URL for product_id {product_id}: {url}. Skipping.")
            continue

        img = download_and_process_image(url)

        if img is None:
            # The warning is printed inside download_and_process_image
            continue

        try:
            img_t = preprocess(img)
            batch_t = torch.unsqueeze(img_t, 0).to(DEVICE)

            with torch.inference_mode():
                features = model(batch_t)

            vector = features.squeeze().cpu().numpy()
            product_vectors[product_id] = vector
            processed_count += 1

            if embedding_dim is None:
                embedding_dim = vector.shape[0]

        except Exception as e:
            print(f"Warning: Failed to generate vector for product_id {product_id} from URL {url}: {e}")
        
    print(f"Image processing complete. Successfully generated vectors for {processed_count} images.")

    if not product_vectors:
        print("Error: No features were extracted from any image. Check warning logs. Exiting.")
        exit()

    if embedding_dim is None:
        print("Error: Embedding dimension could not be determined (no images processed successfully). Exiting.")
        exit()

    num_embeddings = MAX_PRODUCT_ID + 1
    embedding_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=None)

    embedding_layer.weight.data.zero_()

    vectors_loaded = 0
    for product_id, vector in product_vectors.items():
        if isinstance(product_id, (int, np.integer)) and 0 <= product_id < num_embeddings:
            embedding_layer.weight.data[product_id] = torch.tensor(vector)
            vectors_loaded += 1
        else:
            print(f"Warning: Product ID {product_id} (type: {type(product_id)}) is invalid or out of range [0, {MAX_PRODUCT_ID}]. Skipping vector assignment.")

    print(f"nn.Embedding layer weights initialized with {vectors_loaded} vectors.")

    torch.save(embedding_layer.state_dict(), EMBEDDING_SAVE_PATH)
    print(f"Embedding layer state_dict saved to file: '{EMBEDDING_SAVE_PATH}'")
    ```